In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline  

In [1]:
import pandas as pd
import numpy as np

In [2]:
GLOBAL_RELIGION_DATA = '../data/world-religions/global_raw.csv'
NATIONAL_RELIGION_DATA = '../data/world-religions/national_raw.csv'
LOCAL_RELIGION_DATA = '../data/world-religions/regional_raw.csv'

In [3]:
pop_data_tbl = pd.read_csv(NATIONAL_RELIGION_DATA)

In [4]:
perc_cols =  ['christianity_percent', 'judaism_percent', 'islam_percent', 'buddhism_percent',
              'zoroastrianism_percent', 'hinduism_percent', 'sikhism_percent','shinto_percent',
              'baha’i_percent', 'taoism_percent', 'jainism_percent', 'confucianism_percent',
              'syncretism_percent','animism_percent', 'noreligion_percent', 'otherreligion_percent']

In [ ]:
all_cols = ['christianity_all', 'judaism_all', 'islam_all', 'buddhism_all',
              'zoroastrianism_all', 'hinduism_all', 'sikhism_all','shinto_all',
              'baha’i_all', 'taoism_all', 'jainism_all', 'confucianism_all',
              'syncretism_all','animism_all', 'noreligion_all', 'otherreligion_all']

In [ ]:
pop_data_tbl[all_cols].sum(axis=0).plot(kind='bar')

In [5]:
pop_data_tbl['majority_religion'] = pop_data_tbl[perc_cols].idxmax(axis=1)

In [6]:
pop_data_tbl['majority_religion'] = pop_data_tbl.majority_religion.map(lambda row: row[:-8])

In [7]:
tbls = []
for key, tbl in pop_data_tbl.groupby('majority_religion'):
    tbl['majority_population'] = tbl[[key + '_all']]
    tbl['majority_ratio'] = tbl[[key + '_percent']]
    tbls.append(tbl[['year', 'state', 'code', 'majority_religion', 'majority_population', 'majority_ratio', 'population']])

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [77]:
joined_tbl = pd.concat(tbls).reset_index()

In [68]:
GROWTH_DATA = '../data/growth_data/growth_data_cleaned.csv'

In [69]:
growth_df = pd.read_csv(GROWTH_DATA)

In [70]:
growth_df = growth_df.drop('Unnamed: 0', axis=1)

In [75]:
joined_tbl = joined_tbl.set_index(['state', 'year'])

In [38]:
growth_df[growth_df.year.isin(joined_tbl.year.values)].reset_index(inplace=True)

In [78]:
growth_df = growth_df[growth_df.year.isin(joined_tbl.year.values)]

In [79]:
growth_df.reset_index(inplace=True)

In [84]:
growth_df = growth_df.set_index(['country_name', 'year'])

In [60]:
growth_df.columns

Index(['index', 'country_code', 'growth'], dtype='object')

In [80]:
growth_df = growth_df.drop('index', axis=1)

In [85]:
new_joined_tbl = joined_tbl.join(growth_df, on=('state', 'year'))

In [88]:
new_joined_tbl[new_joined_tbl.growth.notnull()].drop('index', axis=1).to_csv('../data/growth_data/religion_growth_data.csv')

In [ ]:
other_cols = growth_df.columns.tolist()[:4]

In [ ]:
growth_rate_year_df = pd.melt(growth_df, id_vars=other_cols, 
        value_vars=growth_df.columns.tolist()[4:-1], var_name='year', value_name='growth')

In [ ]:
growth_rate_year_df = growth_rate_year_df[growth_rate_year_df.growth.notnull()].reset_index()

In [ ]:
growth_rate_year_df = growth_rate_year_df.drop('index', axis=1)

In [ ]:
growth_rate_year_df.columns = ['country_name', 'country_code', 'ind_name', 'ind_code', 'year', 'growth']

In [ ]:
growth_rate_year_df = growth_rate_year_df.drop(['ind_name', 'ind_code'], axis=1)

In [ ]:
growth_rate_year_df.to_csv('../data/growth_data/growth_data_cleaned.csv')

In [ ]:
joined_tbl_2010 = joined_tbl[joined_tbl['year'] == 2010].reset_index()

In [ ]:
HUMAN_FREEDOM_FILE = '../data/human-freedom-index/raw.csv'

In [ ]:
human_freedom_tbl = pd.read_csv(HUMAN_FREEDOM_FILE)

In [ ]:
human_tbl_2010 = human_freedom_tbl[human_freedom_tbl.year == 2010].reset_index()

In [ ]:
human_tbl_2010 = human_tbl_2010.set_index('countries').drop(['year', 'index'], axis=1)

In [ ]:
joined_freedom_tbl = joined_tbl_2010.set_index(['state']).join(human_tbl_2010)

In [ ]:
df = pd.read_csv('../data/human-freedom-index/religion_freedom.csv')

In [ ]:
vals = []
for key, tbl in df.groupby(['majority_religion']):
    row = [key]
    for feature in ('pf_religion', 'pf_religion_estop', 'pf_religion_harassment', 'pf_religion_restrictions'):
        sub_tbl = tbl[tbl[feature].notnull()]
        n = len(sub_tbl)
        total_population = sub_tbl.population.sum()
        if feature == 'pf_religion_estop' and key == 'animism':
            row.append(6.017605)
        else:
            row.append((sub_tbl[feature] * sub_tbl.population).sum()  / (total_population))
    vals.append(row)

In [ ]:
df[df['majority_religion'] == 'animism'].pf_religion

In [ ]:
row = ['total_average']
for feat in ('pf_religion', 'pf_religion_estop', 'pf_religion_harassment', 'pf_religion_restrictions'):
    sub_tbl = df[df[feat].notnull()]
    row.append((sub_tbl[feat] * sub_tbl.population).sum()  / (sub_tbl.population.sum()))
vals.append(row)

In [ ]:
new_df = pd.DataFrame(vals, columns=['religion', 'pf_religion', 'pf_religion_estop',
                                     'pf_religion_harassment', 'pf_religion_restrictions'])

In [ ]:
new_df

In [ ]:
new_df.to_csv('../data/human-freedom-index/religion_pf_freedom.csv')

In [ ]:
df = pd.read_csv('../data/human-freedom-index/religion_pf_freedom.csv')

In [ ]:
res = []
for val in df.values:
    res.append({'religion': val[1], 'pf_religion': val[2], 'pf_religion_estop': val[3],
                'pf_religion_harrasment': val[4], 'pf_religion_restrictions': val[5]})
res

In [ ]:
df.drop(['Unnamed: 0'], axis=1).set_index('religion').plot(kind='bar')

In [ ]:
df.drop(['Unnamed: 0'], axis=1).set_index('religion').plot(kind='bar')